In [1]:
import numpy as np
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [32]:
news_df = pd.read_csv('train.csv.json')

In [43]:
news_df.head()

,"{""@context"":{""@language"":""en""","@vocab:""https://schema.org/""","citeAs:""cr:citeAs""","column:""cr:column""","conformsTo:""dct:conformsTo""","cr:""http://mlcommons.org/croissant/""","data:{""@id"":""cr:data""","@type:""@json""}","dataBiases:""cr:dataBiases""","dataCollection:""cr:dataCollection""",...,"name:""Kaggle"".1","url:""https://www.kaggle.com/organizations/kaggle""","image:""https://storage.googleapis.com/kaggle-organizations/4/thumbnail.png""}","thumbnailUrl:""https://storage.googleapis.com/kaggle-datasets-images/new-version-temp-images/default-backgrounds-67.png-13209942/dataset-card.png""","dateModified:""2024-04-25T15:21:59.793""","datePublished:""2024-04-25T15:21:59.793""","@type:""sc:Dataset""","name:""Scam Detection | Fake News Labelled Dataset""","url:""https://www.kaggle.com/datasets/noorsaeed/scam-detection-fake-news-labelled-dataset/versions/1""","description:""""}"


# About the Dataset:

id: unique id for a news article                                     
title: the title of a news article                                  
author: author of the news article                                    
text: the text of the article; could be incomplete                                        
label: a label that marks whether the news article is real or fake:                              
    1: Fake news                                                 
    0: real News                                        

# 1 Preprocessing 

In [34]:
news_df.isnull().sum()

{"@context":{"@language":"en"                                                                           0
@vocab:"https://schema.org/"                                                                            0
citeAs:"cr:citeAs"                                                                                      0
column:"cr:column"                                                                                      0
conformsTo:"dct:conformsTo"                                                                             0
                                                                                                       ..
datePublished:"2024-04-25T15:21:59.793"                                                                 0
@type:"sc:Dataset"                                                                                      0
name:"Scam Detection | Fake News Labelled  Dataset"                                                     0
url:"https://www.kaggle.com/datasets/noorsaeed

In [35]:
news_df.shape

(0, 112)

In [36]:
news_df = news_df.fillna(' ')

In [37]:
news_df.isnull().sum()

{"@context":{"@language":"en"                                                                           0
@vocab:"https://schema.org/"                                                                            0
citeAs:"cr:citeAs"                                                                                      0
column:"cr:column"                                                                                      0
conformsTo:"dct:conformsTo"                                                                             0
                                                                                                       ..
datePublished:"2024-04-25T15:21:59.793"                                                                 0
@type:"sc:Dataset"                                                                                      0
name:"Scam Detection | Fake News Labelled  Dataset"                                                     0
url:"https://www.kaggle.com/datasets/noorsaeed

In [63]:
expected_cols = ['author', 'title', 'content']
available_cols = [col for col in expected_cols if col in news_df.columns]

print("Available columns:", available_cols)
print(news_df[available_cols].head())


Available columns: []
Empty DataFrame
Columns: []
Index: []


In [64]:
news_df

,"{""@context"":{""@language"":""en""","@vocab:""https://schema.org/""","citeAs:""cr:citeAs""","column:""cr:column""","conformsTo:""dct:conformsTo""","cr:""http://mlcommons.org/croissant/""","data:{""@id"":""cr:data""","@type:""@json""}","dataBiases:""cr:dataBiases""","dataCollection:""cr:dataCollection""",...,"name:""Kaggle"".1","url:""https://www.kaggle.com/organizations/kaggle""","image:""https://storage.googleapis.com/kaggle-organizations/4/thumbnail.png""}","thumbnailUrl:""https://storage.googleapis.com/kaggle-datasets-images/new-version-temp-images/default-backgrounds-67.png-13209942/dataset-card.png""","dateModified:""2024-04-25T15:21:59.793""","datePublished:""2024-04-25T15:21:59.793""","@type:""sc:Dataset""","name:""Scam Detection | Fake News Labelled Dataset""","url:""https://www.kaggle.com/datasets/noorsaeed/scam-detection-fake-news-labelled-dataset/versions/1""","description:""""}"


# separating the data & label

In [68]:
news_df.columns = news_df.columns.str.strip()



In [70]:
print(news_df.columns.tolist())


['{"@context":{"@language":"en"', '@vocab:"https://schema.org/"', 'citeAs:"cr:citeAs"', 'column:"cr:column"', 'conformsTo:"dct:conformsTo"', 'cr:"http://mlcommons.org/croissant/"', 'data:{"@id":"cr:data"', '@type:"@json"}', 'dataBiases:"cr:dataBiases"', 'dataCollection:"cr:dataCollection"', 'dataType:{"@id":"cr:dataType"', '@type:"@vocab"}', 'dct:"http://purl.org/dc/terms/"', 'extract:"cr:extract"', 'field:"cr:field"', 'fileProperty:"cr:fileProperty"', 'fileObject:"cr:fileObject"', 'fileSet:"cr:fileSet"', 'format:"cr:format"', 'includes:"cr:includes"', 'isEnumeration:"cr:isEnumeration"', 'isLiveDataset:"cr:isLiveDataset"', 'jsonPath:"cr:jsonPath"', 'key:"cr:key"', 'md5:"cr:md5"', 'parentField:"cr:parentField"', 'path:"cr:path"', 'personalSensitiveInformation:"cr:personalSensitiveInformation"', 'recordSet:"cr:recordSet"', 'references:"cr:references"', 'regex:"cr:regex"', 'repeated:"cr:repeated"', 'replace:"cr:replace"', 'sc:"https://schema.org/"', 'separator:"cr:separator"', 'source:"cr

# Stemming:

Stemming is the process of reducing a word to its Root word

example: hung         hanged        hanging ======hang

# Steps:
lower case                 
splitting                             
removing stopwords                              
stemming                                   

In [71]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [73]:
# Clean column names
news_df.columns = news_df.columns.str.strip()

# Check all columns
print(news_df.columns.tolist())

# Then update this line based on actual column name:
# news_df['content'] = news_df['content'].apply(stemming)


['{"@context":{"@language":"en"', '@vocab:"https://schema.org/"', 'citeAs:"cr:citeAs"', 'column:"cr:column"', 'conformsTo:"dct:conformsTo"', 'cr:"http://mlcommons.org/croissant/"', 'data:{"@id":"cr:data"', '@type:"@json"}', 'dataBiases:"cr:dataBiases"', 'dataCollection:"cr:dataCollection"', 'dataType:{"@id":"cr:dataType"', '@type:"@vocab"}', 'dct:"http://purl.org/dc/terms/"', 'extract:"cr:extract"', 'field:"cr:field"', 'fileProperty:"cr:fileProperty"', 'fileObject:"cr:fileObject"', 'fileSet:"cr:fileSet"', 'format:"cr:format"', 'includes:"cr:includes"', 'isEnumeration:"cr:isEnumeration"', 'isLiveDataset:"cr:isLiveDataset"', 'jsonPath:"cr:jsonPath"', 'key:"cr:key"', 'md5:"cr:md5"', 'parentField:"cr:parentField"', 'path:"cr:path"', 'personalSensitiveInformation:"cr:personalSensitiveInformation"', 'recordSet:"cr:recordSet"', 'references:"cr:references"', 'regex:"cr:regex"', 'repeated:"cr:repeated"', 'replace:"cr:replace"', 'sc:"https://schema.org/"', 'separator:"cr:separator"', 'source:"cr

In [75]:
print(news_df.columns.tolist())


['{"@context":{"@language":"en"', '@vocab:"https://schema.org/"', 'citeAs:"cr:citeAs"', 'column:"cr:column"', 'conformsTo:"dct:conformsTo"', 'cr:"http://mlcommons.org/croissant/"', 'data:{"@id":"cr:data"', '@type:"@json"}', 'dataBiases:"cr:dataBiases"', 'dataCollection:"cr:dataCollection"', 'dataType:{"@id":"cr:dataType"', '@type:"@vocab"}', 'dct:"http://purl.org/dc/terms/"', 'extract:"cr:extract"', 'field:"cr:field"', 'fileProperty:"cr:fileProperty"', 'fileObject:"cr:fileObject"', 'fileSet:"cr:fileSet"', 'format:"cr:format"', 'includes:"cr:includes"', 'isEnumeration:"cr:isEnumeration"', 'isLiveDataset:"cr:isLiveDataset"', 'jsonPath:"cr:jsonPath"', 'key:"cr:key"', 'md5:"cr:md5"', 'parentField:"cr:parentField"', 'path:"cr:path"', 'personalSensitiveInformation:"cr:personalSensitiveInformation"', 'recordSet:"cr:recordSet"', 'references:"cr:references"', 'regex:"cr:regex"', 'repeated:"cr:repeated"', 'replace:"cr:replace"', 'sc:"https://schema.org/"', 'separator:"cr:separator"', 'source:"cr

# separating the data and label


In [77]:
print(news_df.columns.tolist())


['{"@context":{"@language":"en"', '@vocab:"https://schema.org/"', 'citeAs:"cr:citeAs"', 'column:"cr:column"', 'conformsTo:"dct:conformsTo"', 'cr:"http://mlcommons.org/croissant/"', 'data:{"@id":"cr:data"', '@type:"@json"}', 'dataBiases:"cr:dataBiases"', 'dataCollection:"cr:dataCollection"', 'dataType:{"@id":"cr:dataType"', '@type:"@vocab"}', 'dct:"http://purl.org/dc/terms/"', 'extract:"cr:extract"', 'field:"cr:field"', 'fileProperty:"cr:fileProperty"', 'fileObject:"cr:fileObject"', 'fileSet:"cr:fileSet"', 'format:"cr:format"', 'includes:"cr:includes"', 'isEnumeration:"cr:isEnumeration"', 'isLiveDataset:"cr:isLiveDataset"', 'jsonPath:"cr:jsonPath"', 'key:"cr:key"', 'md5:"cr:md5"', 'parentField:"cr:parentField"', 'path:"cr:path"', 'personalSensitiveInformation:"cr:personalSensitiveInformation"', 'recordSet:"cr:recordSet"', 'references:"cr:references"', 'regex:"cr:regex"', 'repeated:"cr:repeated"', 'replace:"cr:replace"', 'sc:"https://schema.org/"', 'separator:"cr:separator"', 'source:"cr

# converting the textual data to numerical data

In [98]:
X = news_df['content'].values
y = news_df['label'].values


KeyError: 'content'

In [82]:
print(news_df.columns.tolist())


['{"@context":{"@language":"en"', '@vocab:"https://schema.org/"', 'citeAs:"cr:citeAs"', 'column:"cr:column"', 'conformsTo:"dct:conformsTo"', 'cr:"http://mlcommons.org/croissant/"', 'data:{"@id":"cr:data"', '@type:"@json"}', 'dataBiases:"cr:dataBiases"', 'dataCollection:"cr:dataCollection"', 'dataType:{"@id":"cr:dataType"', '@type:"@vocab"}', 'dct:"http://purl.org/dc/terms/"', 'extract:"cr:extract"', 'field:"cr:field"', 'fileProperty:"cr:fileProperty"', 'fileObject:"cr:fileObject"', 'fileSet:"cr:fileSet"', 'format:"cr:format"', 'includes:"cr:includes"', 'isEnumeration:"cr:isEnumeration"', 'isLiveDataset:"cr:isLiveDataset"', 'jsonPath:"cr:jsonPath"', 'key:"cr:key"', 'md5:"cr:md5"', 'parentField:"cr:parentField"', 'path:"cr:path"', 'personalSensitiveInformation:"cr:personalSensitiveInformation"', 'recordSet:"cr:recordSet"', 'references:"cr:references"', 'regex:"cr:regex"', 'repeated:"cr:repeated"', 'replace:"cr:replace"', 'sc:"https://schema.org/"', 'separator:"cr:separator"', 'source:"cr

# Splitting the dataset to training & test data

In [88]:
print(news_df.columns.tolist())


['{"@context":{"@language":"en"', '@vocab:"https://schema.org/"', 'citeAs:"cr:citeAs"', 'column:"cr:column"', 'conformsTo:"dct:conformsTo"', 'cr:"http://mlcommons.org/croissant/"', 'data:{"@id":"cr:data"', '@type:"@json"}', 'dataBiases:"cr:dataBiases"', 'dataCollection:"cr:dataCollection"', 'dataType:{"@id":"cr:dataType"', '@type:"@vocab"}', 'dct:"http://purl.org/dc/terms/"', 'extract:"cr:extract"', 'field:"cr:field"', 'fileProperty:"cr:fileProperty"', 'fileObject:"cr:fileObject"', 'fileSet:"cr:fileSet"', 'format:"cr:format"', 'includes:"cr:includes"', 'isEnumeration:"cr:isEnumeration"', 'isLiveDataset:"cr:isLiveDataset"', 'jsonPath:"cr:jsonPath"', 'key:"cr:key"', 'md5:"cr:md5"', 'parentField:"cr:parentField"', 'path:"cr:path"', 'personalSensitiveInformation:"cr:personalSensitiveInformation"', 'recordSet:"cr:recordSet"', 'references:"cr:references"', 'regex:"cr:regex"', 'repeated:"cr:repeated"', 'replace:"cr:replace"', 'sc:"https://schema.org/"', 'separator:"cr:separator"', 'source:"cr

In [95]:
print(X_train.shape)


NameError: name 'X_train' is not defined

# Training the Model: Logistic Regression

In [96]:
model = LogisticRegression()
model.fit(X_train,Y_train)

NameError: name 'X_train' is not defined

In [103]:
# on training set
train_y_pred = model.predict(X_train)
print(accuracy_score(train_y_pred,Y_train))

NameError: name 'X_train' is not defined

In [102]:
# on testing set
testing_y_pred = model.predict(X_test)
print(accuracy_score(testing_y_pred,Y_test))

NameError: name 'X_test' is not defined

# Detection System

In [101]:
input_data = X_test[10]
prediction = model.predict(input_data)

NameError: name 'X_test' is not defined

In [100]:
if prediction[0] == 0:
    print('The News Is Real')
else:
    print('The News is Fake')

NameError: name 'prediction' is not defined

In [99]:
news_df['content'][2]

KeyError: 'content'